In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
import pickle
import os

import nlpsig
import nlpsig_networks

from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from nlpsig.classification_utils import DataSplits, Folds
from nlpsig_networks.pytorch_utils import SaveBestModel, training_pytorch, testing_pytorch, set_seed, KFold_pytorch
from nlpsig_networks.focal_loss import FocalLoss
from nlpsig_networks.ffn import FeedforwardNeuralNetModel
from nlpsig_networks.deepsignet import StackedDeepSigNet

from tqdm.notebook import tqdm

seed = 2023

In [2]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    implement_ffn,
    ffn_hyperparameter_search,
    obtain_mean_history,
    obtain_signatures_history,
    histories_baseline_hyperparameter_search
)
from nlpsig_networks.scripts.sdsn_functions import (
    obtain_SDSN_input,
    sdsn_hyperparameter_search
)

In [3]:
import signatory

In [4]:
signatory.signature_channels(channels=16, depth=4)

69904

In [5]:
signatory.logsignature_channels(in_channels=16, depth=4)

17816

In [6]:
output_dir = "client_talk_type"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [7]:
anno_mi = pd.read_csv("AnnoMI-full.csv")
anno_mi["datetime"] = pd.to_datetime(anno_mi["timestamp"])
anno_mi = anno_mi.drop(columns=["video_title", "video_url"])
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-06-07 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-07 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-07 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-07 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-07 00:00:34


In [8]:
anno_mi["interlocutor"].value_counts()

therapist    6826
client       6725
Name: interlocutor, dtype: int64

In [9]:
anno_mi["main_therapist_behaviour"].value_counts() / anno_mi["interlocutor"].value_counts()["therapist"]

other              0.313947
question           0.286258
reflection         0.251538
therapist_input    0.148257
Name: main_therapist_behaviour, dtype: float64

In [10]:
anno_mi["client_talk_type"].value_counts() / anno_mi["interlocutor"].value_counts()["client"]

neutral    0.627063
change     0.248030
sustain    0.124907
Name: client_talk_type, dtype: float64

In [11]:
anno_mi["interlocutor"].value_counts()

therapist    6826
client       6725
Name: interlocutor, dtype: int64

In [12]:
anno_mi["topic"].value_counts()

reducing alcohol consumption                                                          2326
more exercise / increasing activity                                                   2034
reducing recidivism                                                                   1303
reducing drug use                                                                     1104
diabetes management                                                                    948
smoking cessation                                                                      923
smoking cessation                                                                      541
taking medicine / following medical procedure                                          448
asthma management                                                                      431
avoiding DOI                                                                           394
changing approach to disease                                                           315

In [13]:
len(anno_mi["transcript_id"].unique())

133

## Only considering client for now...

In [14]:
client_index = [isinstance(x, str) for x in anno_mi["client_talk_type"]]
sum(client_index)

6725

In [15]:
y_data = anno_mi["client_talk_type"][client_index]
y_data.shape

(6725,)

In [16]:
y_data[0:20]

1     neutral
3     neutral
5     neutral
7     neutral
9     neutral
11    neutral
13    neutral
15    neutral
17    neutral
19    neutral
21    neutral
23    neutral
25    neutral
27    neutral
29    neutral
31    neutral
33    neutral
35     change
37     change
39     change
Name: client_talk_type, dtype: object

In [17]:
label_to_id = {y_data.unique()[i]: i for i in range(len(y_data.unique()))}
id_to_label = {v: k for k, v in label_to_id.items()}

In [18]:
label_to_id

{'neutral': 0, 'change': 1, 'sustain': 2}

In [19]:
id_to_label

{0: 'neutral', 1: 'change', 2: 'sustain'}

In [20]:
y_data = [label_to_id[x] for x in y_data]
y_data[0:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]

In [21]:
output_dim = len(label_to_id.values())
output_dim

3

## Obtaining SBERT Embeddings

We can use the `SentenceEncoder` class within `nlpsig` to obtain sentence embeddings from a model. This class uses the [`sentence-transformer`](https://www.sbert.net/docs/package_reference/SentenceTransformer.html) package and here, we have use the pre-trained `all-MiniLM-L12-v2` model by passing this name as a string to the class - alternative models can be found [here](https://www.sbert.net/docs/pretrained_models.html).

We can pass our dataframe and the column name which stores the sentences that we wish to encode along with the model name into the constructor of the class to initialise our sentence encoder as follows:

In [22]:
# initialise the Text Encoder
sentence_encoder = nlpsig.SentenceEncoder(df=anno_mi,
                                          feature_name="utterance_text",
                                          model_name="all-MiniLM-L12-v2")

We used the `.load_pretrained_model()` method to load in the pre-trained model - this may require you to download the model if this is the first time running the notebook.

In [23]:
sentence_encoder.load_pretrained_model()

We can then obtain embeddings via the `.obtain_embeddings()` method.

In [24]:
sbert_embeddings = sentence_encoder.obtain_embeddings()

[INFO] number of sentences to encode: 13551


Batches:   0%|          | 0/212 [00:00<?, ?it/s]

In [25]:
sbert_embeddings.shape

(13551, 384)

We can save our embeddings for use later:

In [26]:
np.save(f"{output_dir}/anno_mi_client_sentence_embeddings_384",
        sbert_embeddings)

In [22]:
sbert_embeddings = np.load(f"{output_dir}/anno_mi_client_sentence_embeddings_384.npy")

# Baseline: FFN baseline

Using the embeddings for the sentences directly in a FFN to predict the client talk type.

Going to try out some variations (1 hidden layer, 2 hidden layers and 3 hidden layers - all of size 100)

In [36]:
num_epochs = 100
hidden_dim_sizes = [[100]*i for i in range(1, 4)]
dropout_rates = [0.5, 0.2, 0.1]
learning_rates = [5e-3, 1e-3, 5e-4, 1e-4, 1e-5]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [37]:
hidden_dim_sizes

[[100], [100, 100], [100, 100, 100]]

In [38]:
learning_rates

[0.005, 0.001, 0.0005, 0.0001, 1e-05]

In [39]:
ffn_current, best_ffn_current, _, __ = ffn_hyperparameter_search(
    num_epochs=num_epochs,
    x_data=sbert_embeddings[client_index],
    y_data=y_data,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_current_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in ffn_current_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in ffn_current_focal_2_best_model.csv


In [40]:
ffn_current

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id
0,focal,0.684015,0.618947,"[0.7808471454880296, 0.5281899109792284, 0.547...",0.564228,0.694238,0.607537,"[0.7920646583394563, 0.5475285171102662, 0.483...",[100],0.5,0.00500,0,2,False,0
0,focal,0.692937,0.605870,"[0.7964912280701756, 0.5169628432956381, 0.504...",0.574431,0.726766,0.623467,"[0.821629213483146, 0.5737704918032787, 0.475]",[100],0.5,0.00500,1,2,False,0
0,focal,0.695911,0.619387,"[0.7940652818991099, 0.529032258064516, 0.5350...",0.668241,0.706320,0.613244,"[0.804332129963899, 0.5559999999999999, 0.4794...",[100],0.5,0.00500,12,2,False,0
0,focal,0.698141,0.611627,"[0.79976717112922, 0.5218855218855218, 0.51322...",0.542071,0.713755,0.611268,"[0.8147622427253371, 0.5546218487394958, 0.464...",[100],0.5,0.00500,123,2,False,0
0,focal,0.696654,0.618950,"[0.793575252825699, 0.5481239804241436, 0.5151...",0.681275,0.716543,0.625144,"[0.8127259580621836, 0.5720000000000001, 0.490...",[100],0.5,0.00500,1234,2,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.645353,0.531895,"[0.7853403141361256, 0.3394833948339484, 0.470...",0.592595,0.650558,0.522043,"[0.7971530249110321, 0.3333333333333333, 0.435...","[100, 100, 100]",0.1,0.00001,0,2,False,44
0,focal,0.596283,0.387553,"[0.7388016288539849, 0.42385786802030456, 0.0]",0.758998,0.602230,0.386229,"[0.7382840663302092, 0.4204018547140649, 0.0]","[100, 100, 100]",0.1,0.00001,1,2,False,44
0,focal,0.647584,0.418361,"[0.7968036529680366, 0.45827814569536424, 0.0]",0.643662,0.671933,0.431289,"[0.8022284122562674, 0.4916387959866221, 0.0]","[100, 100, 100]",0.1,0.00001,12,2,False,44
0,focal,0.645353,0.413831,"[0.796149490373726, 0.44534412955465585, 0.0]",0.639678,0.675651,0.429970,"[0.8079834824501033, 0.48192771084337344, 0.0]","[100, 100, 100]",0.1,0.00001,123,2,False,44


In [41]:
best_ffn_current

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold
0,focal,0.699628,0.616654,"[0.7990654205607477, 0.525963149078727, 0.5249...",0.632668,0.722119,0.627415,"[0.8181174805378627, 0.5546218487394958, 0.509...",[100],0.5,0.001,0,2,False
0,focal,0.704833,0.627111,"[0.7983490566037734, 0.5552050473186121, 0.527...",0.681163,0.714684,0.622294,"[0.8117394416607016, 0.5490196078431373, 0.506...",[100],0.5,0.001,1,2,False
0,focal,0.704089,0.625025,"[0.8009395184967704, 0.5366666666666666, 0.537...",0.568209,0.726766,0.635061,"[0.8222698072805139, 0.5684210526315789, 0.514...",[100],0.5,0.001,12,2,False
0,focal,0.694424,0.618451,"[0.7903130537507383, 0.5288461538461539, 0.536...",0.608452,0.724907,0.633952,"[0.8160919540229885, 0.5896414342629483, 0.496...",[100],0.5,0.001,123,2,False
0,focal,0.698885,0.624533,"[0.7961859356376638, 0.5333333333333333, 0.544...",0.541399,0.716543,0.627569,"[0.8112798264642083, 0.5714285714285714, 0.5]",[100],0.5,0.001,1234,2,False


In [42]:
best_ffn_current["f1"].mean()

0.6223547220370722

In [68]:
np.stack(best_ffn_current["f1_scores"]).mean(axis=0)

array([0.7969706 , 0.53600287, 0.5340907 ])

In [43]:
ffn_current_kfold, best_ffn_current_kfold, _, __ = ffn_hyperparameter_search(
    num_epochs=num_epochs,
    x_data=sbert_embeddings[client_index],
    y_data=y_data,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_current_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in ffn_current_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in ffn_current_focal_2_kfold_best_model.csv


In [44]:
ffn_current_kfold

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id
0,focal,0.681784,0.594099,"[0.7857559836544076, 0.505226480836237, 0.4913...",0.545414,0.686374,0.583469,"[0.7960155911650064, 0.5026455026455026, 0.451...",[100],0.5,0.00500,0,2,True,0
1,focal,0.686989,0.582050,"[0.7977272727272727, 0.4909747292418772, 0.457...",0.534665,0.692005,0.589859,"[0.7959183673469388, 0.49382716049382713, 0.47...",[100],0.5,0.00500,0,2,True,0
2,focal,0.682528,0.575796,"[0.788538681948424, 0.5342237061769616, 0.4046...",0.552925,0.700450,0.604532,"[0.7991323210412147, 0.5532467532467532, 0.461...",[100],0.5,0.00500,0,2,True,0
3,focal,0.706320,0.613436,"[0.805491990846682, 0.5451263537906137, 0.4896...",0.577055,0.701014,0.605842,"[0.800520381613183, 0.5459317585301838, 0.4710...",[100],0.5,0.00500,0,2,True,0
4,focal,0.672862,0.596099,"[0.7738814993954052, 0.5225225225225226, 0.491...",0.548656,0.679617,0.605924,"[0.775735294117647, 0.5420353982300885, 0.5]",[100],0.5,0.00500,0,2,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.653532,0.552813,"[0.7872340425531915, 0.3870967741935484, 0.484...",0.625054,0.626126,0.505284,"[0.7734855136084283, 0.3289817232375979, 0.413...","[100, 100, 100]",0.1,0.00001,1234,2,True,44
1,focal,0.640149,0.521224,"[0.7870370370370371, 0.3321917808219178, 0.444...",0.619201,0.636824,0.524574,"[0.7733450241122314, 0.37055214723926383, 0.42...","[100, 100, 100]",0.1,0.00001,1234,2,True,44
2,focal,0.626766,0.499733,"[0.7731188971855256, 0.34596375617792424, 0.38...",0.617926,0.627815,0.508627,"[0.771168041684759, 0.31865284974093266, 0.436...","[100, 100, 100]",0.1,0.00001,1234,2,True,44
3,focal,0.610409,0.487177,"[0.7679719462302746, 0.2956810631229236, 0.397...",0.638740,0.630068,0.509474,"[0.7740511915269197, 0.3742405832320777, 0.380...","[100, 100, 100]",0.1,0.00001,1234,2,True,44


In [45]:
best_ffn_current_kfold

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold
0,focal,0.684758,0.607160,"[0.7886815171583383, 0.5196078431372549, 0.513...",0.553060,0.680180,0.593967,"[0.7824529991047449, 0.5286236297198538, 0.470...",[100],0.5,0.0001,0,2,True
1,focal,0.677323,0.586921,"[0.7866108786610878, 0.5255023183925811, 0.448...",0.531541,0.688626,0.606592,"[0.7841726618705036, 0.5529953917050692, 0.482...",[100],0.5,0.0001,0,2,True
2,focal,0.685502,0.579362,"[0.79133409350057, 0.5272108843537414, 0.41954...",0.550432,0.690315,0.598250,"[0.791083916083916, 0.530718954248366, 0.47294...",[100],0.5,0.0001,0,2,True
3,focal,0.692193,0.602759,"[0.7938931297709922, 0.5475409836065575, 0.466...",0.582319,0.683559,0.591700,"[0.7836153161175423, 0.5513126491646777, 0.440...",[100],0.5,0.0001,0,2,True
4,focal,0.665428,0.588839,"[0.7696019300361883, 0.4969135802469136, 0.5]",0.547304,0.675113,0.592574,"[0.7785356980445657, 0.5266821345707655, 0.472...",[100],0.5,0.0001,0,2,True
0,focal,0.681784,0.600001,"[0.7883124627310675, 0.5033783783783784, 0.508...",0.560043,0.674550,0.580560,"[0.7842786958463601, 0.5160493827160494, 0.441...",[100],0.5,0.0001,1,2,True
1,focal,0.686989,0.596568,"[0.7936132465996452, 0.5281250000000001, 0.467...",0.535979,0.694820,0.612573,"[0.7891699955614737, 0.5463071512309496, 0.502...",[100],0.5,0.0001,1,2,True
2,focal,0.680297,0.573057,"[0.7853042479908152, 0.5385878489326765, 0.395...",0.552261,0.693131,0.605103,"[0.7892888498683055, 0.548469387755102, 0.4775...",[100],0.5,0.0001,1,2,True
3,focal,0.689963,0.602343,"[0.7903699354081034, 0.5454545454545455, 0.471...",0.581351,0.682432,0.587900,"[0.7850799289520426, 0.5445783132530121, 0.434...",[100],0.5,0.0001,1,2,True
4,focal,0.664684,0.587020,"[0.7681246255242661, 0.4929356357927786, 0.5]",0.547436,0.685248,0.604926,"[0.7860696517412936, 0.5317647058823529, 0.496...",[100],0.5,0.0001,1,2,True


In [46]:
best_ffn_current_kfold["f1"].mean()

0.5924049773807214

In [67]:
np.stack(best_ffn_current_kfold["f1_scores"]).mean(axis=0)

array([0.78498169, 0.52167665, 0.47055659])

# Baseline: Averaging history and use FFN

Here, we will use `nlpsig` to construct some paths of embeddings for the last $k$ utterances which we will average and use those in a FFN. We will also concatenate the current utterance embedding to the FFN input - all of this is done in the `obtain_mean_history` function in `nlpsig-networks.scripts.ffn_baseline_functions` which we imported earlier on.

Here, we will run the hyperparameter search to implement the FFN with the same parameters as above. For this baseline, we can also see how well the model performance is affected by the size of the history window $k$.

In [26]:
window_sizes = [5, 10, 20, 50]

In [28]:
ffn_mean_history, best_ffn_mean_history, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_mean_history_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/4 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type/ffn_mean_history_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type/ffn_mean_history_focal_2_best_model.csv


In [29]:
ffn_mean_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id,k,input_dim
0,focal,0.713011,0.614313,"[0.8114157806379407, 0.5272727272727272, 0.504...",0.697381,0.725836,0.625205,"[0.818368745716244, 0.5365853658536585, 0.5206...",[100],0.5,0.00500,0,2,False,0.00,5,766
0,focal,0.710037,0.620470,"[0.8068181818181818, 0.5251798561151079, 0.529...",0.502735,0.717472,0.611772,"[0.8177408177408177, 0.5253863134657837, 0.492...",[100],0.5,0.00500,1,2,False,0.00,5,766
0,focal,0.667658,0.595767,"[0.7693259121830551, 0.5345622119815667, 0.483...",0.602109,0.700743,0.621087,"[0.7975921745673438, 0.5708955223880597, 0.494...",[100],0.5,0.00500,12,2,False,0.00,5,766
0,focal,0.697398,0.627495,"[0.7888161808447353, 0.538961038961039, 0.5547...",0.534903,0.716543,0.626636,"[0.812545587162655, 0.5786407766990291, 0.4887...",[100],0.5,0.00500,123,2,False,0.00,5,766
0,focal,0.709294,0.610480,"[0.8129251700680273, 0.5212765957446809, 0.497...",0.517703,0.731413,0.629395,"[0.8272033310201249, 0.552915766738661, 0.5080...",[100],0.5,0.00500,1234,2,False,0.00,5,766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.617100,0.265933,"[0.7633163501621121, 0.03448275862068966, 0.0]",0.756498,0.640335,0.267512,"[0.7808764940239044, 0.021660649819494584, 0.0]","[100, 100, 100]",0.1,0.00001,0,2,False,3.44,50,766
0,focal,0.667658,0.566020,"[0.7959905660377359, 0.43717728055077454, 0.46...",0.560264,0.693309,0.585954,"[0.8141720896601591, 0.48945147679324896, 0.45...","[100, 100, 100]",0.1,0.00001,1,2,False,3.44,50,766
0,focal,0.617100,0.254406,"[0.7632183908045977, 0.0, 0.0]",0.788310,0.639405,0.260163,"[0.7804878048780487, 0.0, 0.0]","[100, 100, 100]",0.1,0.00001,12,2,False,3.44,50,766
0,focal,0.683271,0.577781,"[0.801648028251913, 0.5021645021645021, 0.4295...",0.594105,0.697955,0.576923,"[0.8155619596541788, 0.51520572450805, 0.4]","[100, 100, 100]",0.1,0.00001,123,2,False,3.44,50,766


In [30]:
best_ffn_mean_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,k,input_dim
0,focal,0.707807,0.631739,"[0.798329355608592, 0.5847589424572317, 0.5121...",0.718380,0.736989,0.648432,"[0.8244719592134013, 0.6247619047619047, 0.496...",[100],0.5,0.001,0,2,False,50,766
0,focal,0.708550,0.635621,"[0.8004807692307692, 0.5795275590551181, 0.526...",0.549926,0.732342,0.639982,"[0.8253968253968255, 0.5984251968503937, 0.496...",[100],0.5,0.001,1,2,False,50,766
0,focal,0.709294,0.633258,"[0.7988269794721409, 0.5654281098546042, 0.535...",0.615128,0.732342,0.644632,"[0.8202166064981948, 0.6078431372549019, 0.505...",[100],0.5,0.001,12,2,False,50,766
0,focal,0.710037,0.632813,"[0.8046783625730994, 0.5382059800664452, 0.555...",0.541037,0.736989,0.641692,"[0.8297567954220315, 0.6012024048096193, 0.494...",[100],0.5,0.001,123,2,False,50,766
0,focal,0.699628,0.631056,"[0.7921760391198045, 0.5714285714285714, 0.529...",0.556330,0.719331,0.640200,"[0.8095952023988006, 0.6074074074074074, 0.503...",[100],0.5,0.001,1234,2,False,50,766


In [31]:
best_ffn_mean_history["f1"].mean()

0.6328974621408594

In [69]:
np.stack(best_ffn_mean_history["f1_scores"]).mean(axis=0)

array([0.7988983 , 0.56786983, 0.53192425])

In [32]:
ffn_mean_history_kfold, best_ffn_mean_history_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_mean_history_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/4 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type/ffn_mean_history_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type/ffn_mean_history_focal_2_kfold_best_model.csv


In [33]:
ffn_mean_history_kfold

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id,k,input_dim
0,focal,0.672862,0.601383,"[0.7743119266055046, 0.5278276481149013, 0.502...",0.536458,0.664977,0.581384,"[0.772706732941708, 0.4993141289437586, 0.4721...",[100],0.5,0.00500,0,2,True,0.00,5,766
1,focal,0.689219,0.590228,"[0.7997678467788741, 0.5151515151515151, 0.455...",0.528168,0.699887,0.612343,"[0.7982456140350876, 0.5367088607594938, 0.502...",[100],0.5,0.00500,0,2,True,0.00,5,766
2,focal,0.687732,0.565361,"[0.791759465478842, 0.5346869712351945, 0.3696...",0.533283,0.695946,0.592782,"[0.7955706984667802, 0.5286458333333333, 0.454...",[100],0.5,0.00500,0,2,True,0.00,5,766
3,focal,0.687732,0.585681,"[0.7934537246049661, 0.4970873786407767, 0.466...",0.567295,0.701577,0.594639,"[0.8050739957716702, 0.5205091937765206, 0.458...",[100],0.5,0.00500,0,2,True,0.00,5,766
4,focal,0.661710,0.585731,"[0.7608173076923077, 0.5103011093502379, 0.486...",0.545385,0.689752,0.612503,"[0.7840858292355833, 0.5374233128834355, 0.516]",[100],0.5,0.00500,0,2,True,0.00,5,766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.678067,0.581589,"[0.7988165680473372, 0.47694753577106525, 0.46...",0.603217,0.668919,0.557512,"[0.7909854175872735, 0.48687350835322196, 0.39...","[100, 100, 100]",0.1,0.00001,1234,2,True,3.44,50,766
1,focal,0.673606,0.563899,"[0.8027923211169284, 0.4462809917355372, 0.442...",0.598215,0.673423,0.574711,"[0.7879592740150508, 0.4847058823529411, 0.451...","[100, 100, 100]",0.1,0.00001,1234,2,True,3.44,50,766
2,focal,0.642379,0.518177,"[0.7772020725388602, 0.4147909967845659, 0.362...",0.603113,0.662162,0.542741,"[0.7956427015250545, 0.42493638676844786, 0.40...","[100, 100, 100]",0.1,0.00001,1234,2,True,3.44,50,766
3,focal,0.640149,0.538257,"[0.7726190476190476, 0.41627543035993736, 0.42...",0.621480,0.650338,0.533928,"[0.7822222222222224, 0.4493597206053551, 0.370...","[100, 100, 100]",0.1,0.00001,1234,2,True,3.44,50,766


In [34]:
best_ffn_mean_history_kfold

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,k,input_dim
0,focal,0.699628,0.629848,"[0.7965998785670917, 0.5466034755134281, 0.546...",0.535128,0.685811,0.601814,"[0.7854578096947935, 0.543030303030303, 0.4769...",[100],0.5,0.0001,0,2,True,20,766
1,focal,0.681041,0.596610,"[0.7869249394673123, 0.5405405405405405, 0.462...",0.507187,0.696509,0.624477,"[0.7877401646843549, 0.5691609977324262, 0.516...",[100],0.5,0.0001,0,2,True,20,766
2,focal,0.689219,0.591243,"[0.789838337182448, 0.5564924114671164, 0.4273...",0.534393,0.692005,0.600451,"[0.7948831054256728, 0.537966537966538, 0.4685...",[100],0.5,0.0001,0,2,True,20,766
3,focal,0.680297,0.601317,"[0.7790487658037327, 0.5457364341085271, 0.479...",0.557449,0.685248,0.597495,"[0.7831541218637994, 0.5563380281690141, 0.452...",[100],0.5,0.0001,0,2,True,20,766
4,focal,0.667658,0.600250,"[0.7642770352369381, 0.5081723625557206, 0.528...",0.528262,0.681869,0.606892,"[0.77910174152154, 0.5415730337078652, 0.5]",[100],0.5,0.0001,0,2,True,20,766
0,focal,0.689219,0.615304,"[0.7893462469733656, 0.5358851674641149, 0.520...",0.531559,0.680743,0.594290,"[0.7827260458839406, 0.5410628019323671, 0.459...",[100],0.5,0.0001,1,2,True,20,766
1,focal,0.680297,0.597075,"[0.7845503922751961, 0.5407066052227342, 0.465...",0.509566,0.685811,0.613716,"[0.7775239835541344, 0.5565819861431871, 0.507...",[100],0.5,0.0001,1,2,True,20,766
2,focal,0.675836,0.578892,"[0.7780373831775701, 0.5499181669394436, 0.408...",0.530567,0.685811,0.598425,"[0.7876344086021505, 0.5445665445665446, 0.463...",[100],0.5,0.0001,1,2,True,20,766
3,focal,0.684758,0.604041,"[0.7838651414810355, 0.5588697017268446, 0.469...",0.553035,0.681306,0.594199,"[0.7785778577857785, 0.5623529411764706, 0.441...",[100],0.5,0.0001,1,2,True,20,766
4,focal,0.667658,0.601386,"[0.7639060568603213, 0.5289747399702823, 0.511...",0.529611,0.674550,0.603511,"[0.7723880597014926, 0.5430167597765363, 0.495...",[100],0.5,0.0001,1,2,True,20,766


In [35]:
best_ffn_mean_history_kfold["f1"].mean()

0.6024489022123928

In [70]:
np.stack(best_ffn_mean_history_kfold["f1_scores"]).mean(axis=0)

array([0.78204441, 0.54354018, 0.48176212])

# Baseline: FFN using signatures

First, we dimension reduce these and then take signatures. We use the path signature as input to the FFN for classification.

We want to choose a dimension and signature depth such that the number of terms in the signature is _roughly_ 384 so that it is comparable to the number of features that we used for the previous baseline where we computed the mean of the history. Again, we are concatenating the features we obtain with the current utterance embedding.

In [71]:
dim_reduce_methods = ["gaussian_random_projection", "umap"]

## Using signature

In [100]:
signature_dimensions_and_sig_depths = [(19, 2), (7, 3), (4, 4), (3, 5)]
[signatory.signature_channels(channels, depth)
 for (channels, depth) in signature_dimensions_and_sig_depths]

[380, 399, 340, 363]

### Using UMAP

In [ ]:
ffn_signature_umap, best_ffn_signature_umap, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    log_signature=False,
    dim_reduce_methods=["umap"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_signature_umap_focal_{gamma}.csv",
    verbose=False
)

In [ ]:
best_ffn_signature_umap["f1"].mean()

In [ ]:
np.stack(best_ffn_signature_umap["f1_scores"]).mean(axis=0)

In [ ]:
ffn_signature_umap_kfold, best_ffn_signature_umap_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    log_signature=False,
    dim_reduce_methods=["umap"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_signature_umap_focal_{gamma}_kfold.csv",
    verbose=False
)

In [ ]:
best_ffn_signature_umap_kfold["f1"].mean()

In [ ]:
np.stack(best_ffn_signature_umap_kfold["f1_scores"]).mean(axis=0)

### Using random projections

In [ ]:
ffn_signature_grp, best_ffn_signature_grp, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    log_signature=False,
    dim_reduce_methods=["gaussian_random_projection"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_signature_grp_focal_{gamma}.csv",
    verbose=False
)

In [ ]:
best_ffn_signature_grp["f1"].mean()

In [ ]:
np.stack(best_ffn_signature_grp["f1_scores"]).mean(axis=0)

In [ ]:
ffn_signature_grp_kfold, best_ffn_signature_grp_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    log_signature=False,
    dim_reduce_methods=["gaussian_random_projection"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_signature_grp_focal_{gamma}_kfold.csv",
    verbose=False
)

In [ ]:
best_ffn_signature_grp_kfold["f1"].mean()

In [ ]:
np.stack(best_ffn_signature_grp_kfold["f1_scores"]).mean(axis=0)

## Using log signature

In [111]:
log_signature_dimensions_and_sig_depths = [(27, 2), (10, 3), (6, 4), (4, 5)]
[signatory.logsignature_channels(channels, depth)
 for (channels, depth) in log_signature_dimensions_and_sig_depths]

[378, 385, 406, 294]

### Using UMAP

In [ ]:
ffn_logsignature_umap, best_ffn_logsignature_umap, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    log_signature=True,
    dim_reduce_methods=["umap"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_umap_focal_{gamma}.csv",
    verbose=False
)

In [ ]:
best_ffn_logsignature_umap["f1"].mean()

In [ ]:
np.stack(best_ffn_logsignature_umap["f1_scores"]).mean(axis=0)

In [ ]:
ffn_logsignature_umap_kfold, best_ffn_logsignature_umap_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    log_signature=True,
    dim_reduce_methods=["umap"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_umap_focal_{gamma}_kfold.csv",
    verbose=False
)

In [ ]:
best_ffn_logsignature_umap_kfold["f1"].mean()

In [ ]:
np.stack(best_ffn_logsignature_umap_kfold["f1_scores"]).mean(axis=0)

### Using random projections

In [ ]:
ffn_logsignature_grp, best_ffn_logsignature_grp, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    log_signature=True,
    dim_reduce_methods=["gaussian_random_projection"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_grp_focal_{gamma}.csv",
    verbose=False
)

In [ ]:
best_ffn_logsignature_grp["f1"].mean()

In [ ]:
np.stack(best_ffn_logsignature_grp["f1_scores"]).mean(axis=0)

In [ ]:
ffn_logsignature_grp_kfold, best_ffn_logsignature_grp_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    log_signature=True,
    dim_reduce_methods=["gaussian_random_projection"],
    dimension_and_sig_depths=signature_dimensions_and_sig_depths,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_grp_focal_{gamma}_kfold.csv",
    verbose=False
)

In [ ]:
best_ffn_logsignature_grp_kfold["f1"].mean()

In [ ]:
np.stack(best_ffn_logsignature_grp_kfold["f1_scores"]).mean(axis=0)

# Baseline: LSTM classification

# Baseline: Fine-tune BERT for classification

# StackedDeepSigNet

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [244]:
time_features = ["time_encoding", "timeline_index"]
path_specifics = {"pad_by": "history",
                  "zero_padding": True,
                  "method": "k_last",
                  "k": 10,
                  "time_feature": time_features,
                  "standardise_method": ["minmax", None],
                  "embeddings": "dim_reduced",
                  "include_current_embedding": True,
                  "pad_from_below": False}

In [298]:
sig_depths = [2,3]
dim_reduce_methods = ["gaussian_random_projection", "umap"]

In [297]:
output_dim = len(label_to_id)
lstm_hidden_dims = [[8,8], [12,12,8]]
num_time_features = len(time_features)
conv_output_channels = [20, 10, 5]
learning_rate = 1e-4

In [300]:
embedding_dim = 384
dimensions = [embedding_dim, 100, 50, 30]